In [1]:


from random import randint
from functools import reduce
def k_means_cluster(image_values, k=3, initial_means=None):
    """
    Separate the provided RGB values into
    k separate clusters using the k-means algorithm,
    then return an updated version of the image
    with the original values replaced with
    the corresponding cluster values.
    
    params:
    image_values = numpy.ndarray[numpy.ndarray[numpy.ndarray[float]]]
    k = int
    initial_means = numpy.ndarray[numpy.ndarray[float]] or None
    
    returns:
    updated_image_values = numpy.ndarray[numpy.ndarray[numpy.ndarray[float]]]
    """
 
    x = image_values.reshape(-1,image_values.shape[-1])
    # Initialize inital_means if necessary
    if initial_means is None:
        initial_means = x[np.random.choice(x.shape[0], size=k, replace=False)]
    u = initial_means
    dist = np.empty(x.shape[:1] + (k,))
    prev_r = None
    while True:
        # Compute sum of squares distance
        for j in range(0,k):
            dist[:,j] = np.sum((x[:]-u[j])**2, axis=1)
        # Compute min sum of square distance and r
        min_dist = np.min(dist, axis=1, out=None, keepdims=True)
        r = dist
        r[r == min_dist] = 1
        r[r != 1] = 0
        # Check for convergence
        if (not prev_r is None) and (r == prev_r).all():
            break
        prev_r = np.copy(r)
        # Compute u
        for j in range(0,k):
            if np.sum(r[:,j]) == 0:
                u[j] = [0,0,0]
            else:
                u[j] = np.sum(r[:,j].reshape(r.shape[:1] + (1,))*x[:], axis=0)/np.sum(r[:,j], axis=0)
    # Construct segmented image
    r = r.astype(int)
    updated_image_values = np.empty(image_values.shape)
    for index in np.ndindex(updated_image_values.shape[:2]):
        updated_image_values[index] = np.array(u[np.argmax(r[index[0]*updated_image_values.shape[1] + index[1]])])
    return updated_image_values



In [2]:
def k_means_test():
    """
    Testing your implementation
    of k-means on the segmented
    bird_color_24 reference images.
    """
    k_min = 2
    k_max = 6
    image_dir = 'images/'
    image_name = 'bird_color_24.png'
    image_values = image_to_matrix(image_dir + image_name)
    # initial mean for each k value
    initial_means = [
        np.array([[0.90980393,0.8392157,0.65098041],[0.83137256,0.80784315,0.69411767]]),
        np.array([[0.90980393,0.8392157,0.65098041],[0.83137256,0.80784315,0.69411767],[0.67450982,0.52941179,0.25490198]]),
        np.array([[0.90980393,0.8392157,0.65098041],[0.83137256,0.80784315,0.69411767],[0.67450982,0.52941179,0.25490198],[0.86666667,0.8392157,0.70588237]]),
        np.array([[0.90980393,0.8392157,0.65098041],[0.83137256,0.80784315,0.69411767],[0.67450982,0.52941179,0.25490198],[0.86666667,0.8392157,0.70588237],[0,0,0]]),
        np.array([[0.90980393,0.8392157,0.65098041],[0.83137256,0.80784315,0.69411767],[0.67450982,0.52941179,0.25490198],[0.86666667,0.8392157,0.70588237],[0,0,0],[0.8392157,0.80392158,0.63921571]]),
    ]
    # test different k values to find best
    for k in range(k_min, k_max+1):
        updated_values = k_means_cluster(image_values, k, initial_means[k-k_min])
        ref_image = image_dir + 'k%d_%s'%(k, image_name)
        ref_values = image_to_matrix(ref_image)
        dist = image_difference(updated_values, ref_values)
        print('Image distance = %.2f'%(dist))
        if(int(dist) == 0):
            print('Clustering for %d clusters produced a realistic image segmentation.'%(k))
        else:
            print('Clustering for %d clusters didn\'t produce a realistic image segmentation.'%(k))

In [3]:


def default_convergence(prev_likelihood, new_likelihood, conv_ctr, conv_ctr_cap=10):
    """
    Default condition for increasing
    convergence counter: 
    new likelihood deviates less than 10%
    from previous likelihood.

    params:
    prev_likelihood = float
    new_likelihood = float
    conv_ctr = int
    conv_ctr_cap = int

    returns:
    conv_ctr = int
    converged = boolean
    """
    increase_convergence_ctr = (abs(prev_likelihood) * 0.9 < 
                                abs(new_likelihood) < 
                                abs(prev_likelihood) * 1.1)
    
    if increase_convergence_ctr:
        conv_ctr+=1
    else:
        conv_ctr =0
        
    return conv_ctr, conv_ctr > conv_ctr_cap



In [4]:
from random import randint
import math
from scipy.misc import logsumexp
class GaussianMixtureModel:
    """
    A Gaussian mixture model
    to represent a provided 
    grayscale image.
    """
    
    def __init__(self, image_matrix, num_components, means=None):
        """
        Initialize a Gaussian mixture model.
        
        params:
        image_matrix = (grayscale) numpy.nparray[numpy.nparray[float]]
        num_components = int
        """
        self.image_matrix = image_matrix
        self.num_components = num_components
        if(means is None):
            self.means = [0]*num_components
        else:
            self.means = means
        self.variances = [0]*num_components
        self.mixing_coefficients = [0]*num_components
    
    def joint_prob(self, val):
        """Calculate the joint 
        log probability of a greyscale
        value within the image.
        
        params:
        val = float
        
        returns:
        joint_prob = float
        """
        means = np.array(self.means)
        variances = np.array(self.variances)
        mixing = np.array(self.mixing_coefficients)
        joint_prob = mixing*(1.0/np.sqrt(2.0*variances*math.pi))*np.exp(-1.0*((val - means)**2/(2.0*variances)))
        
        return math.log(np.sum(joint_prob))
    
    def initialize_training(self):
        """
        Initialize the training
        process by setting each
        component mean to a random
        pixel's value (without replacement),
        each component variance to 1, and
        each component mixing coefficient
        to a uniform value 
        (e.g. 4 components -> [0.25,0.25,0.25,0.25]).
        
        NOTE: this should be called before 
        train_model() in order for tests
        to execute correctly.
        """
        x = self.image_matrix.flatten()
        self.means = x[np.random.choice(x.shape[0], size=self.num_components, replace=False)].tolist()
        self.variances[:] = [1.0] * self.num_components
        self.mixing_coefficients[:] = [1.0/self.num_components] * self.num_components
    
    def train_model(self, convergence_function=default_convergence):
        """
        Train the mixture model 
        using the expectation-maximization
        algorithm. Since each Gaussian is
        a combination of mean and variance,
        this will fill self.means and 
        self.variances, plus 
        self.mixing_coefficients, with
        the values that maximize
        the overall model likelihood.
        
        params:
        convergence_function = function that returns True if convergence is reached
        """
        self.initialize_training()
        log_likelihood = self.likelihood()
        x = self.image_matrix.flatten()
        means = np.array(self.means)
        variances = np.array(self.variances)
        mixing = np.array(self.mixing_coefficients)
        count = 0
        convergence = False
        while not convergence:
            # E step
            resp = np.empty((self.num_components, x.shape[0]))
            for k in range(0, self.num_components):
                resp[k] = mixing[k]*(1.0/np.sqrt(2.0*variances[k]*math.pi))*np.exp(-1.0*((x - means[k])**2/(2.0*variances[k])))
            resp = resp/np.sum(resp, axis=0)
            resp = resp.T
            # M step
            means = np.sum(resp*x[:, np.newaxis], axis=0)/np.sum(resp, axis=0)
            variances = np.sum(resp*np.square(x[:, np.newaxis] - means), axis=0)/np.sum(resp, axis=0)
            mixing = np.sum(resp, axis=0)/resp.shape[0]
            self.means = means
            self.variances = variances
            self.mixing_coefficients = mixing
            # Evaluate likelihood and check for convergence
            prev_likelihood = log_likelihood
            log_likelihood = self.likelihood()
            count, convergence = convergence_function(prev_likelihood, 
                                                      log_likelihood, 
                                                      count)
    
    def segment(self):
        """
        Using the trained model, 
        segment the image matrix into
        the pre-specified number of 
        components. Returns the original 
        image matrix with the each 
        pixel's intensity replaced 
        with its max-likelihood 
        component mean.
        
        returns:
        segment = numpy.ndarray[numpy.ndarray[float]]
        """
        x = self.image_matrix
        means = np.array(self.means)
        variances = np.array(self.variances)
        mixing = np.array(self.mixing_coefficients)
        segment = np.empty(x.shape)
        for index in np.ndindex(segment.shape):
            likelihoods = mixing*(1.0/np.sqrt(2.0*variances*math.pi))*np.exp(-1.0*((x[index] - means)**2/(2.0*variances)))
            max_likelihood = np.argmax(likelihoods)
            segment[index] = means[max_likelihood]
        return segment
    
    def likelihood(self):
        """Assign a log 
        likelihood to the trained
        model based on the following 
        formula for posterior probability:
        ln(Pr(X | mixing, mean, stdev)) = sum((n=1 to N),ln(sum((k=1 to K), mixing_k * N(x_n | mean_k, stdev_k) )))
        
        returns:
        log_likelihood = float [0,1]
        """
        means = np.array(self.means)
        variances = np.array(self.variances)
        mixing = np.array(self.mixing_coefficients)
        joint_probs = np.empty((self.num_components, x.shape[0]))
        for k in range(0, self.num_components):
            joint_probs[k] = mixing[k]*(1.0/np.sqrt(2.0*variances[k]*math.pi))*np.exp(-1.0*((x - means[k])**2/(2.0*variances[k])))
        joint_probs = np.sum(joint_probs, axis=0)
        joint_probs = np.log(joint_probs)
        log_likelihood = np.sum(joint_probs)
        return log_likelihood
        
    def best_segment(self, iters):
        """Determine the best segmentation
        of the image by repeatedly 
        training the model and 
        calculating its likelihood. 
        Return the segment with the
        highest likelihood.
        
        params:
        iters = int
        
        returns:
        segment = numpy.ndarray[numpy.ndarray[float]]
        """
        # finish this
        likelihoods = []
        segments = []
        for i in range(0, iters):
            self.train_model()
            likelihoods.append(self.likelihood())
            segments.append(self.segment())
        segment =  segments[likelihoods.index(max(likelihoods))]
        return segment


def gmm_likelihood_test():
    """Testing the GMM method
    for calculating the overall
    model probability.
    
    returns:
    likelihood = float
    """
    image_file = 'images/party_spock.png'
    image_matrix = image_to_matrix(image_file)
    num_components = 5
    gmm = GaussianMixtureModel(image_matrix, num_components)
    gmm.initialize_training()
    gmm.means = [0.4627451, 0.10196079, 0.027450981, 0.011764706, 0.1254902]
    likelihood = gmm.likelihood()
    return likelihood



In [5]:
def gmm_joint_prob_test():
    """Testing the GMM method
    for calculating the joint 
    log probability of a given point.
    Should return -0.98196.
    
    returns:
    joint_prob = float
    """
    image_file = 'images/party_spock.png'
    image_matrix = image_to_matrix(image_file)
    num_components = 5
    gmm = GaussianMixtureModel(image_matrix, num_components)
    gmm.initialize_training()
    gmm.means = [0.4627451, 0.10196079, 0.027450981, 0.011764706, 0.1254902]
    test_val = 0.4627451
    joint_prob = gmm.joint_prob(0.4627451)
    return joint_prob

In [6]:
def generate_test_mixture(data_size, means, variances, mixing_coefficients):
    """
    Generate synthetic test
    data for a GMM based on
    fixed means, variances and
    mixing coefficients.
    
    params:
    data_size = (int)
    means = [float]
    variances = [float]
    mixing_coefficients = [float]
    
    returns:
    data = np.array[float]
    """

    data = np.zeros(data_size).flatten()

    indices = np.random.choice( len(means), len(data), p=mixing_coefficients)

    for i in range(len(indices)):
        data[i] = np.random.normal(means[indices[i]], variances[indices[i]])

    return np.array([data])